## Transformed Model

In [34]:
import numpy as np
from scipy.optimize import minimize

# ===================== 常量参数：请按实际工艺 / 设计填 =====================
beta_n = 3.9e-3     # nMOS β_n
beta_p = 1.38e-3    # pMOS β_p
lam = 0.05          # 渠长调制 λ
Vth = 0.5           # 阈值电压
A_req = 500.0       # 增益要求
R_L = 1e4           # 负载电阻

# W_i, L_i 如果在这个问题里视为常数，请在这里给出：
# 你可以先随便设一组满足工艺边界的 W, L，用来求 s_i=W_i/L_i
# 这里先举例：全部用某个默认尺寸（你自己改！）
W = np.array([1e-6]*9)  # 占位：9 个晶体管的宽度
L = np.array([1e-6]*9)  # 占位：9 个晶体管的长度

W_min = np.array([12e-9]*9)
W_max = np.array([1e-2]*9)
L_min = np.array([15e-9]*9)
L_max = np.array([1e-2]*9)

# 检查一下常量是否满足边界（调参时自己确认）
assert np.all(W >= W_min) and np.all(W <= W_max)
assert np.all(L >= L_min) and np.all(L <= L_max)

# ===================== 变量拆包 / 装包 =====================

def unpack(x):
    """
    x: 长度 37
    I[0..8], Vov[0..8], dV[0..8], s[0..8], Vdd
    """
    I = x[0:9]
    Vov = x[9:18]
    dV = x[18:27]
    s = x[27:36]
    Vdd = x[36]
    return I, Vov, dV, s, Vdd

# ===================== 目标函数 =====================

def objective(x):
    I, Vov, dV, s, Vdd = unpack(x)
    # 目标: sum_{i in {5,6,8}} I_i * Vdd = I_5 + I_6 + I_8 (注意 0-based index)
    return Vdd * (I[4] + I[5] + I[7])

# ===================== 约束函数 =====================

def constraints_list():
    cons = []

    # 1) W_i / (L_i s_i) = 1   ->  W_i/(L_i*s_i) - 1 = 0
    def make_s_eq(i):
        def fun(x):
            I, Vov, dV, s, Vdd = unpack(x)
            return W[i] / (L[i] * s[i]) - 1.0
        return {'type': 'eq', 'fun': fun}
    for i in range(9):
        cons.append(make_s_eq(i))

    # 2) 器件方程 I_i = 0.5 * β * s_i * Vov_i^2 * (1 + λ(Vov_i + ΔV_i))
    nmos_idx = [0, 1, 4, 5, 6]  # {1,2,5,6,7} -> 0-based
    pmos_idx = [2, 3, 7, 8]     # {3,4,8,9}
    def make_device_eq(i, beta):
        def fun(x):
            I, Vov, dV, s, Vdd = unpack(x)
            rhs = 0.5 * beta * s[i] * (Vov[i]**2) * (1.0 + lam*(Vov[i] + dV[i]))
            return I[i] - rhs
        return {'type': 'eq', 'fun': fun}
    for i in nmos_idx:
        cons.append(make_device_eq(i, beta_n))
    for i in pmos_idx:
        cons.append(make_device_eq(i, beta_p))

    # 3) (I1 + I2) / I6 = 1
    def fun_I12_over_I6(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return (I[0] + I[1]) / I[5] - 1.0
    cons.append({'type': 'eq', 'fun': fun_I12_over_I6})

    # 4) I_i / I_j = 1 for specified current pairs
    equal_pairs = [(8, 4), (0, 2), (1, 3), (6, 7)]
    def make_current_ratio_eq(ii, jj):
        def fun(x):
            I, Vov, dV, s, Vdd = unpack(x)
            return I[ii] / I[jj] - 1.0
        return {'type': 'eq', 'fun': fun}
    for ii, jj in equal_pairs:
        cons.append(make_current_ratio_eq(ii, jj))

    # 5) ΔV_i = V_th, i in {3,5,9} -> indices 2,4,8
    fixed_dV_idx = [2, 4, 8]
    for i in fixed_dV_idx:
        def make_dv_eq(ii):
            def fun(x):
                I, Vov, dV, s, Vdd = unpack(x)
                return dV[ii] - Vth
            return {'type': 'eq', 'fun': fun}
        cons.append(make_dv_eq(i))

    # 6) Vov5 = Vov6 = Vov7  -> indices 4,5,6
    def fun_vov_5_6(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return Vov[4] - Vov[5]
    def fun_vov_6_7(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return Vov[5] - Vov[6]
    cons.append({'type': 'eq', 'fun': fun_vov_5_6})
    cons.append({'type': 'eq', 'fun': fun_vov_6_7})

    #    Vov1 = Vov2  -> indices 0,1
    def fun_vov_1_2(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return Vov[0] - Vov[1]
    cons.append({'type': 'eq', 'fun': fun_vov_1_2})

    #    Vov3 = Vov4  -> indices 2,3
    def fun_vov_3_4(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return Vov[2] - Vov[3]
    cons.append({'type': 'eq', 'fun': fun_vov_3_4})

    # 7) 几个含 Vdd 的电压关系（全部写成 eq）
    # 2(Vov2+Vov4+ΔV2+ΔV4)/Vdd = 1
    def fun_eq1(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return 2*(Vov[1] + Vov[3] + dV[1] + dV[3]) / Vdd - 1.0
    cons.append({'type': 'eq', 'fun': fun_eq1})

    # (Vov7+Vov8+ΔV7+ΔV8)/Vdd = 1
    def fun_eq2(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return (Vov[6] + Vov[7] + dV[6] + dV[7]) / Vdd - 1.0
    cons.append({'type': 'eq', 'fun': fun_eq2})

    # 2(Vov1+Vov6+ΔV6+Vth)/Vdd = 1
    def fun_eq3(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return 2*(Vov[0] + Vov[5] + dV[5] + Vth) / Vdd - 1.0
    cons.append({'type': 'eq', 'fun': fun_eq3})

    # 2(ΔV2+Vov8)/Vdd = 1
    def fun_eq4(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return 2*(dV[1] + Vov[7]) / Vdd - 1.0
    cons.append({'type': 'eq', 'fun': fun_eq4})

    # 2(ΔV1+Vov3)/Vdd = 1
    def fun_eq5(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return 2*(dV[0] + Vov[2]) / Vdd - 1.0
    cons.append({'type': 'eq', 'fun': fun_eq5})

    # 8) Vdd/12 <= 1  ->  1 - Vdd/12 >= 0
    def fun_vdd_ineq(x):
        I, Vov, dV, s, Vdd = unpack(x)
        return 1.0 - Vdd/12.0
    cons.append({'type': 'ineq', 'fun': fun_vdd_ineq})

    # 9) 增益约束:
    # A_req(λI2+λI4)(λI7+λI8+1/RL)/(β_n β_p s1 s8 Vov1 Vov8) <= 1
    # -> 1 - 上式 >= 0
    def fun_gain_ineq(x):
        I, Vov, dV, s, Vdd = unpack(x)
        num = A_req * (lam*I[1] + lam*I[3]) * (lam*I[6] + lam*I[7] + 1.0/R_L)
        den = beta_n * beta_p * s[0] * s[7] * Vov[0] * Vov[7]
        val = num / den
        return 1.0 - val
    cons.append({'type': 'ineq', 'fun': fun_gain_ineq})

    return cons

# ===================== 初始值 & 边界 =====================

# 简单初始化：全部设成正的常数
# 你可以多试几组不同的初始值（非常重要！）
I0 = np.ones(9) * 1e-5
Vov0 = np.ones(9) * 0.1
dV0 = np.ones(9) * Vth
s0 = W / L          # 初始 s 直接设成 W/L
Vdd0 = 1.2

x0 = np.concatenate([I0, Vov0, dV0, s0, np.array([Vdd0])])

# 每个变量的下界设成一个小正数，避免除零
eps = 1e-6
bounds = [(eps, None)] * len(x0)

# ===================== 求解 =====================

cons = constraints_list()

res = minimize(
    objective,
    x0,
    method='SLSQP',
    bounds=bounds,
    constraints=cons,
    options={'maxiter': 5000, 'ftol': 1e-9, 'disp': True}
)

print("status:", res.status, res.message)
print("optimal objective (power):", res.fun)
I_opt, Vov_opt, dV_opt, s_opt, Vdd_opt = unpack(res.x)
print("Vdd_opt =", Vdd_opt)
print("I_opt =", I_opt)
print("Vov_opt =", Vov_opt)
print("dV_opt =", dV_opt)
print("s_opt =", s_opt)

# 计算增益约束中的 val 以便检查
gain_val = (
    (beta_n * beta_p * s_opt[0] * s_opt[7] * Vov_opt[0] * Vov_opt[7]) / (lam * I_opt[1] + lam * I_opt[3]) / (lam * I_opt[6] + lam * I_opt[7] + 1.0 / R_L)
    
)
print("gain constraint val =", gain_val)


Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 7.211894185850803e-05
            Iterations: 19
            Function evaluations: 580
            Gradient evaluations: 15
status: 8 Positive directional derivative for linesearch
optimal objective (power): 7.211894185850803e-05
Vdd_opt = 12.000000000005429
I_opt = [1.01139011e-06 1.00924983e-06 1.00214078e-06 1.00000000e-06
 1.39596209e-06 1.97473681e-06 2.64739260e-06 2.63921292e-06
 1.40565286e-06]
Vov_opt = [0.02228968 0.02228968 0.04213547 0.04213547 0.02947202 0.02947202
 0.02947202 0.06815118 0.05339232]
dV_opt = [5.95786462e+00 5.93184880e+00 5.00000000e-01 4.10781800e-01
 5.00000000e-01 5.85529429e+00 1.19023757e+01 1.00000000e-06
 5.00000000e-01]
s_opt = [1. 1. 1. 1. 1. 1. 1. 1. 1.]
gain constraint val = 811.6533210396357


## Original Model

In [36]:
import numpy as np
from scipy.optimize import minimize

# ========== 22nm 工艺 & 电路参数 ==========
beta_n = 3.9e-3     # A/V^2
beta_p = 1.38e-3    # A/V^2
lam    = 0.05       # 1/V
Vth    = 0.5        # V
R_L    = 10e3       # Ohm
A_req  = 500.0      # 最小增益要求

# 尺寸边界（来自 PDK）
W_min = 12e-9       # 12 nm
W_max = 10e-3       # 10 mm
L_min = 15e-9       # 15 nm
L_max = 10e-3       # 10 mm

eps = 1e-9          # 防止除零


# ========== 变量打包 / 解包 ==========
# x 向量结构：
# [0:9)   -> W_i, i=1..9
# [9:18)  -> L_i, i=1..9
# [18:45) -> V_ik, i=1..9, k=1..3   (按 (i,k) 展开)
# [45:54) -> I_i = I_{i,31}, i=1..9
# [54]    -> Vdd
#
# 下标映射：i_LTX = 1..9  →  python 下标 idx = i_LTX-1
def unpack(x):
    W   = x[0:9]
    L   = x[9:18]
    V   = x[18:45].reshape((9, 3))   # V[i,0]=V_i1; V[i,1]=V_i2; V[i,2]=V_i3
    I31 = x[45:54]
    Vdd = x[54]
    return W, L, V, I31, Vdd


# ========== 目标函数 ==========
# min  Σ_{i∈{5,6,8}} I_{i,31} Vdd
# {5,6,8} → python 索引 {4,5,7}
def objective(x):
    _, _, _, I31, Vdd = unpack(x)
    power = (I31[4] + I31[5] + I31[7]) * Vdd
    return power


# ========== 约束集合 ==========
constraints = []

# ---------- (1) Device equations & 饱和区条件 ----------

# nMOS: i ∈ {1,2,5,6,7} → idx ∈ {0,1,4,5,6}
nmos_idx = [0, 1, 4, 5, 6]
# pMOS: i ∈ {3,4,8,9} → idx ∈ {2,3,7,8}
pmos_idx = [2, 3, 7, 8]


# 1.a nMOS & pMOS 的 I_{i,31} 等式
def device_eq(x):
    W, L, V, I31, _ = unpack(x)
    eqs = []

    s = W / (L + eps)  # W_i / L_i

    # nMOS
    for idx in nmos_idx:
        V1 = V[idx, 0]
        V2 = V[idx, 1]
        V3 = V[idx, 2]
        model = 0.5 * beta_n * s[idx] * (V2 - V1 - Vth) ** 2 * (1.0 + lam * V3 - lam * V1)
        eqs.append(I31[idx] - model)  # = 0

    # pMOS
    for idx in pmos_idx:
        V1 = V[idx, 0]
        V2 = V[idx, 1]
        V3 = V[idx, 2]
        model = 0.5 * beta_p * s[idx] * (V1 - V2 - Vth) ** 2 * (1.0 + lam * V1 - lam * V3)
        eqs.append(I31[idx] - model)

    return np.array(eqs)

constraints.append({
    "type": "eq",
    "fun": device_eq
})


# 1.b 饱和区不等式
# Vi2 - Vi1 ≥ Vth   (nMOS)
def sat_n_Vov(x):
    _, _, V, _, _ = unpack(x)
    vals = []
    for idx in nmos_idx:
        V1 = V[idx, 0]
        V2 = V[idx, 1]
        vals.append((V2 - V1) - Vth)  # ≥ 0
    return np.array(vals)

constraints.append({
    "type": "ineq",
    "fun": sat_n_Vov
})


# Vi1 - Vi2 ≥ Vth   (pMOS) :  -(Vi2 - Vi1) ≥ Vth
def sat_p_Vov(x):
    _, _, V, _, _ = unpack(x)
    vals = []
    for idx in pmos_idx:
        V1 = V[idx, 0]
        V2 = V[idx, 1]
        vals.append((V1 - V2) - Vth)  # ≥ 0
    return np.array(vals)

constraints.append({
    "type": "ineq",
    "fun": sat_p_Vov
})


# Vi3 ≥ Vi2 - Vth   (nMOS)
def sat_n_Vds(x):
    _, _, V, _, _ = unpack(x)
    vals = []
    for idx in nmos_idx:
        V2 = V[idx, 1]
        V3 = V[idx, 2]
        vals.append(V3 - (V2 - Vth))  # ≥ 0
    return np.array(vals)

constraints.append({
    "type": "ineq",
    "fun": sat_n_Vds
})


# -Vi3 ≥ -Vi2 - Vth  (pMOS) →  Vi2 + Vth - Vi3 ≥ 0
def sat_p_Vds(x):
    _, _, V, _, _ = unpack(x)
    vals = []
    for idx in pmos_idx:
        V2 = V[idx, 1]
        V3 = V[idx, 2]
        vals.append(V2 + Vth - V3)  # ≥ 0
    return np.array(vals)

constraints.append({
    "type": "ineq",
    "fun": sat_p_Vds
})


# ---------- (2) Physical bounds ----------
# 用 bounds 体现 W/L 的上下限即可，这里不再额外写约束


# ---------- (3) Kirchhoff 电流平衡 & 电压等式 ----------

# Vdd = Vi1,  i ∈ {3,4,8,9} → idx {2,3,7,8}
def eq_Vdd_Vi1(x):
    _, _, V, _, Vdd = unpack(x)
    eqs = []
    for idx in [2, 3, 7, 8]:
        eqs.append(Vdd - V[idx, 0])
    return np.array(eqs)

constraints.append({
    "type": "eq",
    "fun": eq_Vdd_Vi1
})


# Vdd/2 = Vi2, i ∈ {1,2} → idx {0,1}
def eq_Vdd_half_Vi2(x):
    _, _, V, _, Vdd = unpack(x)
    eqs = []
    for idx in [0, 1]:
        eqs.append(Vdd / 2.0 - V[idx, 1])
    return np.array(eqs)

constraints.append({
    "type": "eq",
    "fun": eq_Vdd_half_Vi2
})


# Vi1 = 0, i ∈ {5,6,7} → idx {4,5,6}
def eq_Vi1_zero(x):
    _, _, V, _, _ = unpack(x)
    eqs = []
    for idx in [4, 5, 6]:
        eqs.append(V[idx, 0])  # = 0
    return np.array(eqs)

constraints.append({
    "type": "eq",
    "fun": eq_Vi1_zero
})


# Vi3 = Vi2, i ∈ {3,5,9} → idx {2,4,8}
def eq_Vi3_Vi2(x):
    _, _, V, _, _ = unpack(x)
    eqs = []
    for idx in [2, 4, 8]:
        eqs.append(V[idx, 2] - V[idx, 1])
    return np.array(eqs)

constraints.append({
    "type": "eq",
    "fun": eq_Vi3_Vi2
})


# V52 = V62 = V72   (i=5,6,7, k=2)
def eq_V52_V62_V72(x):
    _, _, V, _, _ = unpack(x)
    v52 = V[4, 1]
    v62 = V[5, 1]
    v72 = V[6, 1]
    return np.array([v52 - v62, v62 - v72])

constraints.append({
    "type": "eq",
    "fun": eq_V52_V62_V72
})


# V63 = V11 = V21
def eq_V63_V11_V21(x):
    _, _, V, _, _ = unpack(x)
    v63 = V[5, 2]
    v11 = V[0, 0]
    v21 = V[1, 0]
    return np.array([v63 - v11, v11 - v21])

constraints.append({
    "type": "eq",
    "fun": eq_V63_V11_V21
})


# V42 = V13 = V32
def eq_V42_V13_V32(x):
    _, _, V, _, _ = unpack(x)
    v42 = V[3, 1]
    v13 = V[0, 2]
    v32 = V[2, 1]
    return np.array([v42 - v13, v13 - v32])

constraints.append({
    "type": "eq",
    "fun": eq_V42_V13_V32
})


# V43 = V23 = V82
def eq_V43_V23_V82(x):
    _, _, V, _, _ = unpack(x)
    v43 = V[3, 2]
    v23 = V[1, 2]
    v82 = V[7, 1]
    return np.array([v43 - v23, v23 - v82])

constraints.append({
    "type": "eq",
    "fun": eq_V43_V23_V82
})


# V73 = V83
def eq_V73_V83(x):
    _, _, V, _, _ = unpack(x)
    return np.array([V[6, 2] - V[7, 2]])

constraints.append({
    "type": "eq",
    "fun": eq_V73_V83
})


# V93 = V53
def eq_V93_V53(x):
    _, _, V, _, _ = unpack(x)
    return np.array([V[8, 2] - V[4, 2]])

constraints.append({
    "type": "eq",
    "fun": eq_V93_V53
})


# I6,31 = I1,31 + I2,31
def eq_I6_I1_I2(x):
    _, _, _, I31, _ = unpack(x)
    return np.array([I31[5] - (I31[0] + I31[1])])

constraints.append({
    "type": "eq",
    "fun": eq_I6_I1_I2
})


# I_i,31 = I_i',31, (i,i') ∈ {(9,5),(1,3),(2,4),(7,8)}
def eq_current_pairs(x):
    _, _, _, I31, _ = unpack(x)
    eqs = []
    pairs = [(8, 4), (0, 2), (1, 3), (6, 7)]
    for i, j in pairs:
        eqs.append(I31[i] - I31[j])
    return np.array(eqs)

constraints.append({
    "type": "eq",
    "fun": eq_current_pairs
})


# V_ik ≥ 0  和  0 ≤ Vdd ≤ 12  用 bounds 实现即可
# 这里再加一个 Vdd ≥ 0 的冗余不等式也可以：
def ineq_Vdd_nonneg(x):
    _, _, _, _, Vdd = unpack(x)
    return np.array([Vdd])  # ≥ 0

constraints.append({
    "type": "ineq",
    "fun": ineq_Vdd_nonneg
})


# ---------- (4) 增益约束 ----------
# Gain = [ βn βp Π_{i∈{1,8}} (Wi/Li)(Vi2 - Vi1 - Vth) ]
#        / [ (λ I2,31 + λ I4,31) ( λ I7,31 + λ I8,31 + 1/R_L ) ]
# 要求 Gain ≥ A_req  →  Gain - A_req ≥ 0
def ineq_gain(x):
    W, L, V, I31, _ = unpack(x)
    s = W / (L + eps)

    # i=1,8 → idx 0,7
    num = beta_n * beta_p
    for idx in [0, 7]:
        V1 = V[idx, 0]
        V2 = V[idx, 1]
        if idx == 0:
            num *= s[idx] * (V2 - V1 - Vth)
        else:
            num *= s[idx] * (V1 - V2 - Vth)

    denom1 = lam * I31[1] + lam * I31[3]           # λI2,31 + λI4,31
    denom2 = lam * I31[6] + lam * I31[7] + 1.0 / R_L
    denom  = (denom1 + eps) * (denom2 + eps)

    gain = num / (denom + eps)
    return np.array([gain - A_req])

constraints.append({
    "type": "ineq",
    "fun": ineq_gain
})


# ========== 变量边界 ==========
bounds = []

# W_i
for _ in range(9):
    bounds.append((W_min, W_max))

# L_i
for _ in range(9):
    bounds.append((L_min, L_max))

# V_ik: 0 ≤ V_ik ≤ 12
for _ in range(9 * 3):
    bounds.append((0.0, 12.0))

# I_i ≥ 0
for _ in range(9):
    bounds.append((0.0, None))

# Vdd: 0 ≤ Vdd ≤ 12
bounds.append((0.0, 12.0))


# ========== 初始点 x0 ==========
x0 = np.zeros(55)

# W_i, L_i 取几何中点（log 尺度中间）
x0[0:9]  = np.sqrt(W_min * W_max)
x0[9:18] = np.sqrt(L_min * L_max)

# V_ik 随便先放在 0.1V 附近
x0[18:45] = 0.1

# I_i 初始 100µA
x0[45:54] = 100e-5

# Vdd 初始 3.3V
x0[54] = 1


# ========== 调用 SLSQP ==========
res = minimize(
    objective,
    x0,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'maxiter': 5000, 'ftol': 1e-9, 'disp': True}
)

def compute_gain(x):
    W, L, V, I31, _ = unpack(x)
    s = W / (L + eps)

    # i=1,8 → idx 0,7
    num = beta_n * beta_p
    for idx in [0, 7]:
        V1 = V[idx, 0]
        V2 = V[idx, 1]
        if idx == 0:
            num *= s[idx] * (V2 - V1 - Vth)
        else:
            num *= s[idx] * (V1 - V2 - Vth)

    denom1 = lam * I31[1] + lam * I31[3]           # λI2,31 + λI4,31
    denom2 = lam * I31[6] + lam * I31[7] + 1.0 / R_L
    denom  = (denom1 + eps) * (denom2 + eps)

    gain = num / (denom + eps)
    return gain

print("Success:", res.success)
print("Message:", res.message)
print("Optimal objective (power):", res.fun)

W_opt, L_opt, V_opt, I_opt, Vdd_opt = unpack(res.x)
print("Vdd* =", Vdd_opt)
print("W*   =", W_opt)
print("L*   =", L_opt)
print("I*   =", I_opt)
print("Gain* =", compute_gain(res.x))



Inequality constraints incompatible    (Exit mode 4)
            Current function value: 4.2139955247303e-12
            Iterations: 6
            Function evaluations: 338
            Gradient evaluations: 6
Success: False
Message: Inequality constraints incompatible
Optimal objective (power): 4.2139955247303e-12
Vdd* = 1.0000227904193522
W*   = [4.02100541e-04 4.65698155e-05 2.80801420e-05 2.80447974e-05
 3.91930334e-05 1.33624278e-04 2.12387539e-04 1.20000000e-08
 3.33680656e-05]
L*   = [1.92840545e-06 1.17574634e-06 1.59567804e-07 1.59558507e-07
 1.32897913e-06 1.41248941e-06 1.74857412e-07 4.61202181e-08
 1.94341243e-06]
I*   = [2.34435698e-12 4.81824952e-21 2.34435696e-12 2.77919761e-21
 1.86954251e-12 2.34435698e-12 6.46791746e-21 2.92287258e-21
 1.86954251e-12]
Gain* = 0.25460154049344835


/opt/anaconda3/envs/pytorch/lib/python3.11/site-packages/scipy/optimize/_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
